In [3]:
import ribopy
from ribopy import Ribo
from functions import get_sequence, get_cds_range_lookup, get_psite_offset
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import pickle
from matplotlib_venn import venn2
from scipy.stats import zscore
from PIL import Image

In [4]:
# Initialize ribo object
ribo_path = './mouse/all.ribo'
ribo_object = Ribo(ribo_path, alias = ribopy.api.alias.apris_human_alias)
alias = True

# Quality control

### Distribution of read lengths 

In [5]:
# Plot length distribution

# Retrieve information 
experiments = ribo_object.experiments
length_dist = ribo_object.get_length_dist(region_name = "CDS")
selected_data = length_dist.loc[:, experiments]
norm_data = selected_data.div(selected_data.sum(axis=0), axis=1) * 100

# Plot
plt.figure()
for experiment in experiments:
    plt.plot(norm_data.index, norm_data[experiment], label=experiment)

plt.xlabel('Read Length', fontsize=16)
plt.ylabel('Frequency (%)', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=16) 
plt.legend(fontsize=16)
# plt.savefig('length_dist.png')

Read lengths to be analyzed were determined to be 25-31 nucleotides.

Experiments to be analyzed were determined to be ['WT_control_A', 'WT_10min_A', 'WT_30min_A', 'WT_1hr_A']

In [7]:
# Initialize variables
min_len = 25
max_len = 31
experiments = ['WT_control_A', 'WT_10min_A', 'WT_30min_A', 'WT_1hr_A']

### Metagene plot

In [8]:
# Plot metagene radius

# Retrieve information
metagene = ribo_object.get_metagene(site_type="start", range_lower = 25, range_upper = 31)
selected_col = [i for i in range(-50,51)]
metagene_selected = metagene[selected_col]
selected_rows = metagene_selected.loc[experiments]
selected_rows = selected_rows.T

# Plot
plt.figure()  
for col in selected_rows.columns:
    plt.plot(selected_rows.index, selected_rows[col], label=col)

plt.xlabel('Position', fontsize=16)
plt.ylabel('Frequency', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.legend(fontsize=16)
# plt.savefig('metagene.png')

P-site was calculated using the function get_psite_offset() in functions.py

# Analysis

### Normalization

Adjusted coverage data of WT_control_A was compiled as adj_coverage_filtered_WT_control_A.pkl using adj_coverage_filtered.py

In [9]:
# Import pickle file
pickle_path = 'adj_coverage_filtered_WT_control_A.pkl'
with open(pickle_path, "rb") as f:
    coverage = pickle.load(f)

In [10]:
# Find z-scores
zscores = {transcript: zscore(coverage) for transcript, coverage in coverage.items() if coverage is not None}

# Find ribosome occupancy scores
occ_scores = {transcript: coverage / np.mean(coverage) for transcript, coverage in coverage.items() if coverage is not None}

In [11]:
# Visualize relationship between two normalization methods

zscores_values = np.concatenate(list(zscores.values()))
occ_scores_values = np.concatenate(list(occ_scores.values()))

# Fit a linear trendline
slope, intercept = np.polyfit(zscores_values.flatten(), occ_scores_values.flatten(), 1)

# Calculate correlation coefficient
correlation_coefficient = np.corrcoef(zscores_values.flatten(), occ_scores_values.flatten())[0, 1]

# Create a scatter plot
plt.figure()
plt.scatter(zscores_values.flatten(), occ_scores_values.flatten(), alpha=0.5)
plt.plot(zscores_values.flatten(), slope * zscores_values.flatten() + intercept, color='red', label=f'Trendline: y = {slope:.2f}x + {intercept:.2f}')
plt.xlabel('Z-scores', fontsize=16)
plt.ylabel('Ribosome Occupancy Scores', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.legend(fontsize=12)
plt.text(0.7, 0.05, f'Correlation Coefficient: {correlation_coefficient:.2f}', transform=plt.gca().transAxes, fontsize=16)
# plt.savefig('norm_comparison.png')

print("Correlation Coefficient:", correlation_coefficient)

Correlation Coefficient: 0.9838822257588958


### Determination of Stall Sites Using Z-Scores vs. Ribosome Occupancy Scores

In [95]:
constant_threshold = 6.5
transcripts = occ_scores.keys()

# Initialize variables to keep track of counts
sum_unique_z = 0
sum_intersection = 0
sum_unique_occ = 0

for transcript in transcripts:
    # Calculate the variable threshold for each transcript
    variable_threshold = np.mean(occ_scores[transcript]) + 1.5 * np.std(occ_scores[transcript]) 
    
    # Get stall sites for z-scores and occupancy scores
    stall_sites_z = np.where(zscores[transcript] > constant_threshold)[0]
    
    stall_sites_occ = np.where(occ_scores[transcript] > variable_threshold)[0]
    # Comment out if modifying occupancy score threshold
    sorted_indices = stall_sites_occ[np.argsort(occ_scores[transcript][stall_sites_occ])[::-1]]
    stall_sites_occ = sorted_indices[:5]
    
    # Calculate unique and intersecting stall sites for each transcript
    unique_z = len(set(stall_sites_z) - set(stall_sites_occ))
    intersection = len(set(stall_sites_z).intersection(stall_sites_occ))
    unique_occ = len(set(stall_sites_occ) - set(stall_sites_z))
    
    if unique_z > 0 or unique_occ > 0:
        print(transcript, variable_threshold, unique_z, intersection, unique_occ)
    
    # Update the sum of counts across transcripts
    sum_unique_z += unique_z
    sum_intersection += intersection
    sum_unique_occ += unique_occ

Map2-204 5.419366962857549 15 5 0
Tuba4a-202 5.8393623555061 2 5 0
Itm2c-201 5.012680338444899 0 4 1
Ptma-201 4.175631061627661 0 1 4
Dbi-201 4.983326974426941 0 1 4
Atp1b1-201 4.948561687022496 0 4 1
Rgs4-201 5.180316330479082 0 2 3
Atp1a2-201 5.795698003781794 11 5 0
Sptan1-203 5.182313397549173 32 5 0
Dnm1-201 5.608204350665705 12 5 0
Chn1-203 6.076619188170188 2 5 0
Atp5g3-202 4.662106692605509 0 3 2
Hnrnpa3-204 5.359999182569686 1 5 0
Slc1a2-202 5.987348532852421 2 5 0
Scg5-201 5.247426509523621 0 3 2
Prnp-201 5.240355013050321 0 4 1
Snap25-201 4.922665273544599 0 2 3
Dstn-201 5.028010355680239 0 4 1
Cst3-201 6.032761444105643 0 1 4
Fkbp1a-201 4.515470405140913 0 1 4
Nnat-202 4.903957784784013 0 2 3
Ywhab-201 6.734428895778821 0 4 1
Gnas-202 5.133408005612939 1 5 0
Eef1a2-201 5.849613514425174 1 5 0
Stmn3-201 4.774559466874918 0 2 3
Stmn2-201 4.914804480130146 0 2 3
Fabp5-201 4.842107742151612 0 2 3
Pfn2-201 6.058761392406486 0 1 4
Gria2-201 6.04894287658894 6 5 0
Mllt11-201 5.152

In [139]:
# Set font size
plt.rcParams['font.size'] = 16

# Plot the Venn diagram
plt.figure()
venn = venn2(subsets=(sum_unique_z, sum_unique_occ, sum_intersection),
             set_labels=('Z-Scores', 'Occupancy Scores'))
plt.text(-0.3, -0.8, f'Constant Threshold = {constant_threshold}', fontsize=16)
plt.savefig('venn.png')

In [42]:
transcript = 'Cpne6-201'
plt.figure()
plt.bar(range(len(coverage[transcript])), coverage[transcript])
plt.xlabel('Nucleotide Position', fontsize=16)
plt.ylabel('Coverage', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=16)

In [43]:
# constant_threshold = 5.0
stall_sites_z = np.where(zscores[transcript] > constant_threshold)[0]
stall_sites_z

array([718, 820, 823, 826, 901])

In [44]:
variable_threshold = np.mean(occ_scores[transcript]) + 1.5 * np.std(occ_scores[transcript]) 
stall_sites_occ = np.where(occ_scores[transcript] > variable_threshold)[0]

scores = occ_scores[transcript]
stall_sites = np.where(scores > variable_threshold)[0]
sorted_indices = stall_sites[np.argsort(scores[stall_sites])[::-1]]
stall_sites_occ = sorted_indices[:5]
stall_sites_occ

array([826, 820, 823, 718, 901])

In [125]:
count_occ = 0
for pos in stall_sites_occ:
    if count_occ == 0:
        plt.axvline(x=pos, color='y', linestyle='--', label='Stall Sites - Occupancy Scores')
        count_occ += 1
    else:
        plt.axvline(x=pos, color='y', linestyle='--')
    
count_z = 0
for pos in stall_sites_z:
    if count_z == 0:
        plt.axvline(x=pos, color='g', linestyle='--', label='Stall Sites - Z-Scores')
        count_z += 1
    else:
        plt.axvline(x=pos, color='g', linestyle='--')
    
intersection = np.intersect1d(stall_sites_occ, stall_sites_z)
# Plotting the intersection
count_int = 0
for pos in intersection:
    if count_int == 0:
        plt.axvline(x=pos, color='r', linestyle='--', label='Intersection')
        count_int += 1
    else:
        plt.axvline(x=pos, color='r', linestyle='--',)

# Adding legend
plt.legend(loc='upper right', fontsize=14)

plt.savefig('coverage_stall_sites.png')

In [129]:
transcripts = occ_scores.keys()
count_z = 0
count_occ = 0
for transcript in transcripts:
    stall_sites_z = np.where(zscores[transcript] > constant_threshold)[0]
    stall_sites_occ = np.where(occ_scores[transcript] > variable_threshold)[0]
    count_z += len(stall_sites_z)
    count_occ += len(stall_sites_occ)

### Constant vs. Percentile-Based Threshold

In [24]:
# Concatenate all z-scores values
zscores_values = np.concatenate(list(zscores.values()))

# Calculate constant threshold
constant_threshold = np.percentile(zscores_values, 99.55)

# Get transcripts
transcripts = occ_scores.keys()

# Initialize variables to keep track of counts
sum_unique_c = 0
sum_intersection = 0
sum_unique_p = 0

count_c = []
count_p = []

for transcript in transcripts:
    # Calculate percentile-based threshold for each transcript
    percentile_threshold = np.percentile(zscores[transcript], 99.55)
    
    # Get stall sites for each method
    stall_sites_c = np.where(zscores[transcript] > constant_threshold)[0]
    stall_sites_p = np.where(zscores[transcript] > percentile_threshold)[0]
    
    # Calculate unique and intersecting stall sites for each transcript
    unique_c = len(set(stall_sites_c) - set(stall_sites_p))
    intersection = len(set(stall_sites_c).intersection(stall_sites_p))
    unique_p = len(set(stall_sites_p) - set(stall_sites_c))
    
    # Update the sum of counts across transcripts
    sum_unique_c += unique_c
    sum_intersection += intersection
    sum_unique_p += unique_p
    
    count_c.append(unique_c)
    count_p.append(unique_p)

In [27]:
t_statistic, p_value = stats.ttest_rel(count_c, count_p)

# Print results
print("Paired t-test Results:")
print("t-statistic:", t_statistic)
print("p-value:", p_value)

Paired t-test Results:
t-statistic: -2.3182877467881333
p-value: 0.02139144278801359


In [19]:
# Venn Diagram

# Set font size
plt.rcParams['font.size'] = 16

# Plot the Venn diagram
plt.figure()
venn = venn2(subsets=(sum_unique_c, sum_unique_p, sum_intersection),
             set_labels=('Constant', 'Percentile'))
plt.savefig('venn_cp.png')

In [126]:
# transcript = 'Atp5g3-202'
transcript = 'Dbi-201'
plt.figure()
plt.bar(range(len(coverage[transcript])), np.clip(zscores[transcript], a_min=0, a_max=None))
plt.xlabel('Nucleotide Position', fontsize=16)
plt.ylabel('Z-Scores', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=16)

# Find stall sites for constant threshold
stall_sites_c = np.where(zscores[transcript] > c_threshold)[0]
# Percentile-based threshold
v_threshold = np.percentile(zscores[transcript], 99.55)
stall_sites_v = np.where(zscores[transcript] > v_threshold)[0]


count_c = 0
for pos in stall_sites_c:
    if count_c == 0:
        plt.axvline(x=pos, color='y', linestyle='--', label='Stall Sites - Constant Threshold',)
        count_c += 1
    else:
        plt.axvline(x=pos, color='y', linestyle='--')
        
count_v = 0
for pos in stall_sites_v:
    if count_v == 0:
        plt.axvline(x=pos, color='g', linestyle='--', label='Stall Sites - Percentile-Based Threshold')
        count_v += 1
    else:
        plt.axvline(x=pos, color='g', linestyle='--')
    
intersection = np.intersect1d(stall_sites_c, stall_sites_v)
# Plotting the intersection
count_int = 0
for pos in intersection:
    if count_int == 0:
        plt.axvline(x=pos, color='r', linestyle='--', label='Intersection')
        count_int += 1
    else:
        plt.axvline(x=pos, color='r', linestyle='--')

# Adding legend
plt.legend(loc='upper left', fontsize=14)
plt.savefig('coverage_stall_sites.png')
Image.open('coverage_stall_sites.png').show()

In [123]:
stall_sites_v

array([280, 283])

In [78]:
# Set font size
plt.rcParams['font.size'] = 16

# Plot the Venn diagram
plt.figure()
venn = venn2(subsets=(sum_unique_c, sum_unique_v, sum_int),
             set_labels=('Constant Threshold = 6.5', '99th Percentile Threshold For Each Transcript'))
# plt.text(-0.3, -0.8, f'Constant Threshold = {constant_threshold}', fontsize=16)
plt.savefig('venn_cv.png')